# Explanatory Visualizations with focus on intra-simulation variation

## Plot of Z (economic) position

In [1]:
from scipy.stats import beta
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output

from indirect_pathway.src.model.indirect_effect import generate_stratification_positions
from indirect_pathway.src.visualization.plots import create_stratification_plot


plot_height = 600

# Initialize the Dash app with bootstrap theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# App layout using bootstrap components
app.layout = html.Div([
    html.H1("Stratification Position Visualization", className="my-4"),

    # Main row containing controls and chart
    dbc.Row([
        # Left column - Controls card
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Controls"),
                dbc.CardBody([
                    # Histogram controls
                    html.H5("Histogram Controls", className="mb-3"),
                    html.Label("Sample Size:"),
                    dcc.Slider(
                        id='sample-size-slider',
                        min=100, max=10000, step=100, value=1000,
                        marks={i: str(i) for i in range(0, 10001, 2000)},
                        className="mb-4"
                    ),
                    html.Label("Proportion Disadvantaged (p):"),
                    dcc.Slider(
                        id='p-slider',
                        min=0.01, max=0.99, step=0.01, value=0.3,
                        marks={i/10: str(i/10) for i in range(0, 11, 2)},
                        className="mb-4"
                    ),

                    # Distribution controls
                    html.H5("Distribution Parameters", className="mb-3 mt-4"),
                    html.Label("Mean Position (Disadvantaged):"),
                    dcc.Slider(
                        id='mu-disadv-slider',
                        min=0.01, max=0.9, step=0.01, value=0.2,
                        marks={i/10: str(i/10) for i in range(0, 10, 2)},
                        className="mb-4"
                    ),
                    html.Label("Position Gap:"),
                    dcc.Slider(
                        id='z-position-gap-slider',
                        min=0, max=0.8, step=0.01, value=0.3,
                        marks={i/10: str(i/10) for i in range(0, 9, 2)},
                        className="mb-4"
                    ),
                    html.Label("Concentration (Disadvantaged):"),
                    dcc.Slider(
                        id='c-disadv-slider',
                        min=1, max=50, step=1, value=20,
                        marks={i: str(i) for i in range(0, 51, 10)},
                        className="mb-4"
                    ),
                    html.Label("Concentration (Advantaged):"),
                    dcc.Slider(
                        id='c-adv-slider',
                        min=1, max=50, step=1, value=20,
                        marks={i: str(i) for i in range(0, 51, 10)},
                        className="mb-4"
                    ),
                ])
            ])
        ], width=3),

        # Right column - Chart
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Visualization"),
                dbc.CardBody([
                    dcc.Graph(
                        id='stratification-plot',
                        style={'height': f'{plot_height}px', 'width': '100%'}
                    )
                ])
            ])
        ], width=9)
    ])
])

@app.callback(
    Output('stratification-plot', 'figure'),
    [Input('sample-size-slider', 'value'),
     Input('p-slider', 'value'),
     Input('mu-disadv-slider', 'value'),
     Input('z-position-gap-slider', 'value'),
     Input('c-disadv-slider', 'value'),
     Input('c-adv-slider', 'value')]
)
def update_graph(sample_size, p, mu_disadv, z_position_gap, c_disadv, c_adv):
    # Generate positions
    positions = generate_stratification_positions(
        p=p,
        mu_disadv=mu_disadv,
        z_position_gap=z_position_gap,
        c_disadv=c_disadv,
        c_adv=c_adv,
        sample_size=sample_size,
    )
    
    return create_stratification_plot(positions, height=plot_height)

In [2]:
# Run the app and display in notebook
from IPython.display import IFrame
app.run(jupyter_mode='external', port=8050)
IFrame(src=f"http://127.0.0.1:8050", width="100%", height=plot_height+200)

Dash app running on http://127.0.0.1:8050/


## Plot of Normalized Expected Incarceration Curves

In [5]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output

from indirect_pathway.src.model.indirect_effect import generate_stratification_positions, calculate_incarceration_rates_normalized
from indirect_pathway.src.visualization.plots import create_incarceration_rate_plot

# Initialize the Dash app with bootstrap theme
app_incarceration_rate = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Set plot height
plot_height = 700

# App layout using bootstrap components
app_incarceration_rate.layout = dbc.Container([
    html.H1("Incarceration Rate Visualization", className="my-4"),

    # Main row containing controls and chart
    dbc.Row([
        # Left column - Controls card
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Controls"),
                dbc.CardBody([
                    # Position distribution controls
                    html.H5("Population Parameters", className="mb-3"),
                    html.Label("Sample Size:"),
                    dcc.Slider(
                        id='sample-size-slider',
                        min=100, max=10000, step=100, value=1000,
                        marks={i: str(i) for i in range(0, 10001, 2000)},
                        className="mb-4"
                    ),
                    html.Label("Proportion Disadvantaged (p):"),
                    dcc.Slider(
                        id='p-slider',
                        min=0.01, max=0.99, step=0.01, value=0.3,
                        marks={i/10: str(i/10) for i in range(0, 11, 2)},
                        className="mb-4"
                    ),
                    
                    # Distribution controls
                    html.H5("Distribution Parameters", className="mb-3 mt-4"),
                    html.Label("Mean Position (Disadvantaged):"),
                    dcc.Slider(
                        id='mu-disadv-slider',
                        min=0.01, max=0.9, step=0.01, value=0.2,
                        marks={i/10: str(i/10) for i in range(0, 10, 2)},
                        className="mb-4"
                    ),
                    html.Label("Position Gap:"),
                    dcc.Slider(
                        id='z-position-gap-slider',
                        min=0, max=0.8, step=0.01, value=0.3,
                        marks={i/10: str(i/10) for i in range(0, 9, 2)},
                        className="mb-4"
                    ),
                    html.Label("Concentration (Disadvantaged):"),
                    dcc.Slider(
                        id='c-disadv-slider',
                        min=1, max=50, step=1, value=20,
                        marks={i: str(i) for i in range(0, 51, 10)},
                        className="mb-4"
                    ),
                    html.Label("Concentration (Advantaged):"),
                    dcc.Slider(
                        id='c-adv-slider',
                        min=1, max=50, step=1, value=20,
                        marks={i: str(i) for i in range(0, 51, 10)},
                        className="mb-4"
                    ),
                    
                    # Incarceration rate parameters
                    html.H5("Incarceration Rate Parameters", className="mb-3 mt-4"),
                    html.Label("Shape Parameter (γ):"),
                    dcc.Slider(
                        id='gamma-slider',
                        min=0, max=5, step=0.1, value=1,
                        marks={i: str(i) for i in range(0, 6)},
                        className="mb-4"
                    ),
                    html.Label("Target Average Rate (per 100,000):"),
                    dcc.Slider(
                        id='target-rate-slider',
                        min=100, max=1000, step=50, value=500,
                        marks={i: str(i) for i in range(100, 1001, 200)},
                        className="mb-4"
                    ),
                ])
            ])
        ], width=3),

        # Right column - Statistics and Chart
        dbc.Col([
            # Statistics card on top
            dbc.Card([
                dbc.CardHeader("Statistics"),
                dbc.CardBody(id='stats-container')
            ], className="mb-4"),
            
            # Visualization card below
            dbc.Card([
                dbc.CardHeader("Visualization"),
                dbc.CardBody([
                    dcc.Graph(
                        id='incarceration-plot',
                        style={'height': f'{plot_height}px', 'width': '100%'}
                    )
                ])
            ])
        ], width=9)
    ])
], fluid=True)

@app_incarceration_rate.callback(
    [Output('incarceration-plot', 'figure'),
     Output('stats-container', 'children')],
    [Input('sample-size-slider', 'value'),
     Input('p-slider', 'value'),
     Input('mu-disadv-slider', 'value'),
     Input('z-position-gap-slider', 'value'),
     Input('c-disadv-slider', 'value'),
     Input('c-adv-slider', 'value'),
     Input('gamma-slider', 'value'),
     Input('target-rate-slider', 'value')]
)
def update_graph(sample_size, p, mu_disadv, z_position_gap, c_disadv, c_adv, gamma, target_avg_rate):
    # Generate positions
    positions = generate_stratification_positions(
        p=p,
        mu_disadv=mu_disadv,
        z_position_gap=z_position_gap,
        c_disadv=c_disadv,
        c_adv=c_adv,
        sample_size=sample_size,
    )
    
    # Calculate incarceration rates
    rate_data = calculate_incarceration_rates_normalized(
        positions=positions,
        gamma=gamma,
        target_avg_rate=target_avg_rate
    )
    
    # Create plot
    fig = create_incarceration_rate_plot(
        rate_data=rate_data,
        gamma=gamma,
        target_avg_rate=target_avg_rate,
        positions=positions,
        height=plot_height
    )
    
    # Create statistics display
    stats = html.Div([
        dbc.Row([
            dbc.Col([
                html.P([
                    html.Strong("Disadvantaged Group Rate: "), 
                    f"{rate_data['rate_disadv']:.1f} per 100,000"
                ]),
                html.P([
                    html.Strong("Advantaged Group Rate: "), 
                    f"{rate_data['rate_adv']:.1f} per 100,000"
                ])
            ], width=4),
            dbc.Col([
                html.P([
                    html.Strong("Population Average Rate: "), 
                    f"{rate_data['pop_avg_rate']:.1f} per 100,000"
                ]),
                html.P([
                    html.Strong("Disparity Ratio: "), 
                    f"{rate_data['rate_disadv'] / rate_data['rate_adv']:.2f}"
                ])
            ], width=4),
            dbc.Col([
                html.P([
                    html.Strong("Disparity Difference: "), 
                    f"{rate_data['rate_disadv'] - rate_data['rate_adv']:.1f} per 100,000"
                ]),
                html.P([
                    html.Strong("Normalized Disparity Index (η): "), 
                    f"{(rate_data['rate_disadv'] / rate_data['rate_adv'] - 1) / (rate_data['rate_disadv'] / rate_data['rate_adv'] + (1-p)/p):.3f}"
                ])
            ], width=4)
        ])
    ])
    
    return fig, stats

In [6]:
# Run the app and display in notebook
from IPython.display import IFrame
app_incarceration_rate.run(jupyter_mode='external', port=8051)
IFrame(src=f"http://127.0.0.1:8051", width="100%", height=plot_height+500)

Dash app running on http://127.0.0.1:8051/
